# Wikipedia Events Database
---
  
**Goal:** create a database for the events I retrieve from Wikipedia and write python scripts for CRUD-ing data in the database.   
  
  
**Notes:** 
- this is a supporting notebook for the TADS_Wikipedia_this_day_in_history project  

### TO-DO:
---  
  
- [ ] create database  
- [ ] create tables  
- [x] create picture of the database schema
- [ ] write code for CRUD-ing data in the database
    - example

In [1]:
import sqlite3

### Preliminary schema

In [2]:
%%html 
<img src = './images/img_ss_database_schema_01_01feb21.jpg' width='70%'>

## Create database and tables

In [ ]:
with sqlite3.connect('wikipedia_tdih.db') as conn:
    # initialize cursor
    c = conn.cursor()
    
    # create table wiki_day_data_log
    c.execute('''CREATE TABLE wiki_day_data_log 
                (day_data_log_id INT AUTOINCREMENT,
                 doe TEXT,
                 day INT,
                 month INT,
                 status_code INT,
                 day_soup BLOB,
                 no_event_for TEXT''')
    
    # create table wiki_link
    c.execute('''CREATE TABLE wiki_link 
                (link_id INT AUTOINCREMENT,
                 doe TEXT,
                 link_url TEXT UNIQUE,
                 PRIMARY KEY (link_id))''')
    
    # create table wiki_event
    c.execute('''CREATE TABLE wiki_event 
                (event_id INT AUTOINCREMENT,
                 doe TEXT,
                 day INT,
                 month INT,
                 year INT,
                 bc_ad TEXT,
                 bc_ad_note TEXT,
                 event_type TEXT,
                 event_description TEXT,
                 event_first_link_id INT,
                 event_links_list BLOB,
                 FOREIGN KEY (event_first_link_id) REFERENCES wiki_link(link_id),
                 PRIMARY KEY (event_id))''')
    

    # create table wiki_link_usage
    c.execute('''CREATE TABLE wiki_link_usage 
                (link_usage_id INT AUTOINCREMENT,
                 doe TEXT,
                 link_id INT,
                 event_id INT,
                 is_first_link INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id),
                 FOREIGN KEY (event_id) REFERENCES wiki_event(event_id)''')
    
    # create table wiki_link_data_log
    c.execute('''CREATE TABLE wiki_link_data_log 
                (link_data_log_id INT AUTOINCREMENT,
                 doe TEXT,
                 link_id INT,
                 status_code INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id)''')
    
    # create table wiki_link_size
    c.execute('''CREATE TABLE wiki_link_size 
                (link_size_id INT AUTOINCREMENT,
                 doe TEXT,
                 link_id INT, 
                 link_size INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id)''')
    
    # create table wiki_link_page_views
    c.execute('''CREATE TABLE wiki_link_page_views 
                (link_page_views_id INT AUTOINCREMENT,
                 doe TEXT,
                 link_id INT,
                 page_views_date TEXT,
                 page_views INT,
                 FOREIGN KEY (link_id) REFERENCES wiki_link(link_id)''')
    
    # create table wiki_image
    c.execute('''CREATE TABLE wiki_image 
                (image_id INT AUTOINCREMENT,
                 doe TEXT,
                 image_url TEXT,
                 PRIMARY KEY (image_id))''')
    
    # create table wiki_image_usage
    c.execute('''CREATE TABLE wiki_image_usage
                (image_usage_id INT AUTOINCREMENT,
                 doe TEXT,
                 image_id INT,
                 link_id INT,
                 FOREIGN KEY (image_id) REFERENCES wiki_image(image_id))''')
    
    # create table wiki_credit
    c.execute('''CREATE TABLE wiki_credit 
                (credit_id INT AUTOINCREMENT,
                 doe TEXT,
                 credit_username TEXT UNIQUE,
                 PRIMARY KEY (credit_id))''')
    
    # create table wiki_copyright license
    c.execute('''CREATE TABLE wiki_copyright_license
                (copyright_license_id INT AUTOINCREMENT,
                 doe TEXT,
                 license_name TEXT,
                 license_description TEXT,
                 is_reuse_rights INT,
                 PRIMARY KEY (copyright_license_id))''')
    
    # create table wiki_image_info
    c.execute('''CREATE TABLE wiki_image_info 
                (image_info_id INT AUTOINCREMENT,
                 doe TEXT,
                 image_id INT,
                 copyright_license_id INT,
                 image_credit_id INT,
                 FOREIGN KEY (image_id) REFERENCES wiki_image(image_id),
                 FOREIGN KEY (copyright_license_id) REFERENCES wiki_copyright_license(copyright_license_id),
                 FOREIGN KEY (image_credit_id) REFERNCES wiki_credit(credit_id))''')
    


Flow:
---  
  
1. run day_data_main
    - returns nested dictionary day_data_dict
        - status_code
        - day_text_soup
        - events
        - births
        - deaths
        - holidays_and_observances
        - no_event_for  


2. log data into wiki_get_day_data_log
    - id
    - day
    - month
    - status_code
    - day_text_soup
    - no_event_for